# 频率和时频传感器分析

目标是展示如何探索你的数据（频率和时频）的谱内容。

In [1]:
import pathlib
import matplotlib

import mne
import mne_bids

matplotlib.use('Qt5Agg')
mne.set_log_level('WARNING')

In [3]:
# 设置参数
sample_data_dir = 'F:/Database/Multimodal_data/mne_data/MNE-sample-data/' # 设置已有数据路径
sample_data_dir = pathlib.Path(sample_data_dir)
epochs = mne.read_epochs(pathlib.Path(sample_data_dir / 'out_data') / 'epochs_epo.fif')

In [26]:
epochs.apply_proj()
epochs_auditory = epochs['Auditory']
epochs_auditory

Number of events,145
Events,Auditory/Left: 72Auditory/Right: 73
Time range,-0.300 – 0.499 s
Baseline,-0.300 – 0.000 s


## 频率分析
首先来探索 epochs 的频率内容

通过跨段平均查看所有通道类型

In [25]:
epochs_auditory.plot_psd(fmin=2., fmax=40., average=True, bandwidth=2)
# 'bandwidth'参数控制着多维度的光谱分辨率。可以通过选择较窄的带宽来提高分辨率，但需要更长的计算时间。

<MNELineFigure size 1000x850 with 3 Axes>

查看PSD的空间分布

In [11]:
epochs_auditory.plot_psd_topomap(ch_type='eeg', normalize=False)

<Figure size 2560x1311 with 10 Axes>

In [12]:
epochs_auditory.plot_psd_topomap(ch_type='mag', normalize=False)

<Figure size 2560x1311 with 10 Axes>

In [13]:
epochs_auditory.plot_psd_topomap(ch_type='grad', normalize=False)

<Figure size 2560x1311 with 10 Axes>

有时，考虑相对功率可能很有趣，相对功率定义为给定频带中的功率除以总功率。要探索此选项，请查看“normalize”关键字。

## 时频分析：功率和试验间一致性

现在，让我们从我们的 Epochs 中计算时频表示（TFRs）。
我们将看看功率和试验间一致性（ITC）。

为此，我们将使用函数 `mne.time_frequency.tfr_morlet`
但是，您也可以使用 `mne.time_frequency.tfr_multitaper`
或 `mne.time_frequency.tfr_stockwell`。

In [32]:
import numpy as np

# 定义感兴趣的频率（对数间隔）
freqs = np.logspace(*np.log10([2, 30]), num=20)
n_cycles = freqs / 2. # 每个频率的不同循环次数
power, itc = mne.time_frequency.tfr_morlet(epochs_auditory, freqs=freqs, n_cycles=n_cycles, use_fft=True, 
                                           return_itc=True, decim=3, n_jobs=1)
power

<AverageTFR | time : [-0.299693, 0.499488], freq : [2.000000, 30.000000], nave : 145, channels : 364, ~9.4 MB>

In [33]:
power.crop(-0.1, 0.7) # 裁剪以去除边缘伪影

C:\Users\pomelo\AppData\Local\Temp\ipykernel_4660\3763878784.py:1: RuntimeWarning: tmax is not in time interval. tmax is set to <class 'mne.time_frequency.tfr.AverageTFR'>.tmax (0.499488 s)
  power.crop(-0.1, 0.7) # 裁剪以去除边缘伪影


<AverageTFR | time : [-0.099898, 0.499488], freq : [2.000000, 30.000000], nave : 145, channels : 364, ~7.2 MB>

In [16]:
itc.crop(-0.1, 0.7)

C:\Users\pomelo\AppData\Local\Temp\ipykernel_4660\3658010723.py:1: RuntimeWarning: tmax is not in time interval. tmax is set to <class 'mne.time_frequency.tfr.AverageTFR'>.tmax (0.499488 s)
  itc.crop(-0.1, 0.7)


<AverageTFR | time : [-0.099898, 0.499488], freq : [2.000000, 30.000000], nave : 145, channels : 364, ~7.2 MB>

### 功率检查
生成的图形是交互式的。在拓扑图中，您可以单击一个图像来可视化一个传感器的数据。您还可以选择时频平面中的一部分，以获得某个时频区域的地形图。

In [17]:
baseline_mode = 'logratio' 
baseline = (None, 0)

### 画出功率拓扑图

In [34]:
(power.copy()
 .pick_types(eeg=True, meg=False)
 .plot_topo())

d:\Miniconda3\envs\MF\lib\site-packages\matplotlib\cbook\__init__.py:314: RuntimeWarning: `mask` is None, not masking the plot ...
  func(*args, **kwargs)
d:\Miniconda3\envs\MF\lib\site-packages\matplotlib\cbook\__init__.py:314: RuntimeWarning: `mask` is None, not adding contour to the plot ...
  func(*args, **kwargs)
d:\Miniconda3\envs\MF\lib\site-packages\matplotlib\cbook\__init__.py:314: RuntimeWarning: `mask` is None, not masking the plot ...
  func(*args, **kwargs)
d:\Miniconda3\envs\MF\lib\site-packages\matplotlib\cbook\__init__.py:314: RuntimeWarning: `mask` is None, not adding contour to the plot ...
  func(*args, **kwargs)
d:\Miniconda3\envs\MF\lib\site-packages\matplotlib\cbook\__init__.py:314: RuntimeWarning: `mask` is None, not masking the plot ...
  func(*args, **kwargs)
d:\Miniconda3\envs\MF\lib\site-packages\matplotlib\cbook\__init__.py:314: RuntimeWarning: `mask` is None, not adding contour to the plot ...
  func(*args, **kwargs)
d:\Miniconda3\envs\MF\lib\site-packages\

<Figure size 2560x1311 with 2 Axes>

### 画出一个单独通道的功率图

In [35]:
power.plot(picks='EEG 050', baseline=baseline, mode=baseline_mode)

[<Figure size 640x480 with 2 Axes>]

### 画出特定频率范围的拓扑图

In [39]:
import matplotlib.pyplot as plt

fig, axis = plt.subplots(1, 3, figsize=(7, 4))
power.plot_topomap(ch_type='grad', tmin=-0.1, tmax=0.5, fmin=4, fmax=7, baseline=baseline, mode=baseline_mode, 
                   axes=axis[0], show=False, contours=1) # 经过前面的裁剪操作，此时的数据段只有-0.1到0.5秒的数据
axis[0].set_title('Theta') # 单独给子图命名
power.plot_topomap(ch_type='grad', tmin=-0.1, tmax=0.5, fmin=8, fmax=12, baseline=baseline, mode=baseline_mode, 
                   axes=axis[1], show=False, contours=1)
axis[1].set_title('Alpha')
power.plot_topomap(ch_type='grad', tmin=-0.1, tmax=0.5, fmin=15, fmax=30, baseline=baseline, mode=baseline_mode, 
                   axes=axis[2], show=False, contours=1)
axis[2].set_title('Beta')
plt.tight_layout()
plt.show()

### 联合作图
还可以创建一个显示聚合 TFR 的联合图，通过通道和拓扑图在特定的时间和频率，以获得关于跨越时间和空间的振荡效应的简要概述。

In [40]:
power.plot_joint(baseline=baseline, mode='mean', tmin=None, tmax=None, timefreqs=[(0.05, 2.), (0.1, 11.)])
plt.show()

### 一致性检验

In [41]:
itc.plot_topo(title='Inter-Trial coherence', vmin=0, vmax=0.5, cmap='Reds')

d:\Miniconda3\envs\MF\lib\site-packages\matplotlib\cbook\__init__.py:314: RuntimeWarning: `mask` is None, not masking the plot ...
  func(*args, **kwargs)
d:\Miniconda3\envs\MF\lib\site-packages\matplotlib\cbook\__init__.py:314: RuntimeWarning: `mask` is None, not adding contour to the plot ...
  func(*args, **kwargs)
d:\Miniconda3\envs\MF\lib\site-packages\matplotlib\cbook\__init__.py:314: RuntimeWarning: `mask` is None, not masking the plot ...
  func(*args, **kwargs)
d:\Miniconda3\envs\MF\lib\site-packages\matplotlib\cbook\__init__.py:314: RuntimeWarning: `mask` is None, not adding contour to the plot ...
  func(*args, **kwargs)
d:\Miniconda3\envs\MF\lib\site-packages\matplotlib\cbook\__init__.py:314: RuntimeWarning: `mask` is None, not masking the plot ...
  func(*args, **kwargs)
d:\Miniconda3\envs\MF\lib\site-packages\matplotlib\cbook\__init__.py:314: RuntimeWarning: `mask` is None, not adding contour to the plot ...
  func(*args, **kwargs)
d:\Miniconda3\envs\MF\lib\site-packages\

<Figure size 2560x1311 with 2 Axes>

## 练习
ITC拓扑图可视化

In [42]:
(itc.copy()
 .pick_types(eeg=True, meg=False)
 .plot_topo())

d:\Miniconda3\envs\MF\lib\site-packages\matplotlib\cbook\__init__.py:314: RuntimeWarning: `mask` is None, not masking the plot ...
  func(*args, **kwargs)
d:\Miniconda3\envs\MF\lib\site-packages\matplotlib\cbook\__init__.py:314: RuntimeWarning: `mask` is None, not adding contour to the plot ...
  func(*args, **kwargs)
d:\Miniconda3\envs\MF\lib\site-packages\matplotlib\cbook\__init__.py:314: RuntimeWarning: `mask` is None, not masking the plot ...
  func(*args, **kwargs)
d:\Miniconda3\envs\MF\lib\site-packages\matplotlib\cbook\__init__.py:314: RuntimeWarning: `mask` is None, not adding contour to the plot ...
  func(*args, **kwargs)
d:\Miniconda3\envs\MF\lib\site-packages\matplotlib\cbook\__init__.py:314: RuntimeWarning: `mask` is None, not masking the plot ...
  func(*args, **kwargs)
d:\Miniconda3\envs\MF\lib\site-packages\matplotlib\cbook\__init__.py:314: RuntimeWarning: `mask` is None, not adding contour to the plot ...
  func(*args, **kwargs)
d:\Miniconda3\envs\MF\lib\site-packages\

<Figure size 2560x1311 with 2 Axes>

In [43]:
itc.plot(picks='EEG 050', baseline=baseline, mode=baseline_mode)

[<Figure size 640x480 with 2 Axes>]

In [44]:
fig, axis = plt.subplots(1, 3, figsize=(7, 4))
itc.plot_topomap(ch_type='grad', tmin=-0.1, tmax=0.5, fmin=4, fmax=7,
                 baseline=baseline, mode=baseline_mode, axes=axis[0],
                 show=False, contours=1)
axis[0].set_title('Theta')
itc.plot_topomap(ch_type='grad', tmin=-0.1, tmax=0.5, fmin=8, fmax=12,
                 baseline=baseline, mode=baseline_mode, axes=axis[1],
                 show=False, contours=1)
axis[1].set_title('Alpha')
itc.plot_topomap(ch_type='grad', tmin=-0.1, tmax=0.5, fmin=15, fmax=30,
                 baseline=baseline, mode=baseline_mode, axes=axis[2],
                 show=False, contours=1)
axis[2].set_title('Beta')
plt.tight_layout()
plt.show()

In [46]:
itc.plot_joint(baseline=baseline, mode='mean', tmin=None, tmax=None, timefreqs=[(0.05, 2.), (0.1, 11.)])
plt.show()